In [ ]:
from files import get_dataset
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np

In [ ]:
def oneR_firstStep(data):
    temp_d = []
    start = 0
    end = 0
    while end < len(data):
        c = data[start][1]
        end = start
        for d in data[start:]:
            if d[1] != c:
                temp_d.append([start, end-1])
                start = end
                break
            end += 1
    return temp_d 

def oneR_secondStep(oner_fd, data, threshold):
    temp_d = []
    while len(oner_fd) > 1:
        zero_count = 0
        one_count = 0
        start = 0
        end = 0
        temp = [-1, 0, 0, 0, 0]
        while zero_count < threshold and one_count < threshold:
            start = oner_fd[0][0]
            if temp[0] == -1:
                temp[0] = start
            end = oner_fd[0][1]
            label = data[start][1]
            if label == 1:
                one_count += end - start
            else:
                zero_count += end - start
            del oner_fd[0]
            if len(oner_fd) == 0:
                break
            
        temp[1] = end
        temp[2] = zero_count
        temp[3] = one_count
        temp_d.append(temp)
    return temp_d

def oneR_thirdStep(oner_ds, data):

    def max_label(index):
        label = max(oner_ds[index][2], oner_ds[index][3])
        index_label = oner_ds[index].index(label)
        if index_label == 2:
            label = 0
        else:
            label = 1

        return label

    temp_d = []
    index = 0
    while index < len(oner_ds)-1:
        label = max_label(index)
        temp = oner_ds[index]
        for i in range(index+1, len(oner_ds)):
            temp_l = max_label(i)
            index = i
            if label == temp_l:
                temp[1] = oner_ds[i][1]
                temp[2] += oner_ds[i][2]
                temp[3] += oner_ds[i][3]
            else:
                break
        temp[4] = data[temp[1]][0]
        temp.append(label)
        temp_d.append(temp)
    
    return temp_d

In [ ]:
def cal_error(oner_dt):
    error_sum = 0
    total = 0
    for d in oner_dt:
        error_sum += min(d[2], d[3])
        total += d[2] + d[3]

    return error_sum / total

def predict_numeric(min_error, x):
    feature = min_error[0]
    for i in min_error[2]:
        if x[feature] < i[4]:
            return i[5]   
        
def predict_nominal(min_error, x):
    pass

In [ ]:
df = get_dataset()
y = df[["cardio"]]
df_num = df.select_dtypes(exclude='object')
df_num_cat = df_num.drop(["smoke", "alco", "active"], axis=1)
df_temp1 = df_num_cat
df_temp2 = df.drop(["age", "height", "weight", "ap_hi", "ap_lo"], axis=1)
print(df_temp2)
error_rate_sum = 0
print(df_num_cat)

In [ ]:
for _ in range(5):
    #numeric
    shuffled = shuffle(df_temp1)
    X_train = shuffled.sample(frac = 0.8)
    X_test = shuffled.drop(X_train.index)
    y_test = X_test[["cardio"]]
    X_test = X_test.drop(["cardio"], axis=1)
    print(X_train)

    errors = []
    oner_results = []
    for i, t in enumerate(["age", "height", "weight", "ap_hi", "ap_lo"]):
        df_temp = X_train[[t, "cardio"]]
        df_temp = df_temp.sort_values(t)
        print(df_temp)
        #print(df_temp["cardio"].value_counts())
        df_temp = df_temp.to_numpy()
        oner_f = oneR_firstStep(df_temp)
        #print(oner_f)
        oner_s = oneR_secondStep(oner_f, df_temp, len(df_temp) // 10)
        #print(oner_s)
        oner_t = oneR_thirdStep(oner_s, df_temp)
        print(oner_t)
        oner_results.append(oner_t)
        errors.append([i, cal_error(oner_t), oner_t])        

    min_error = min(errors, key=lambda x: x[1])
    print("min error:", min_error)

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()
    y_test = np.ravel(y_test)

    error_rate = 0
    for i in range(len(X_test)):
        #print("predicted:", predict(min_error, X_test[i]), "actual:", y_test[i])
        if predict_numeric(min_error, X_test[i]) != y_test[i]:
            error_rate += 1
    error_rate /= len(y_test)
    error_rate_sum += error_rate
    print("error_rate:", error_rate)

    #nominal
#     shuffled = shuffle(df_temp2)
#     X_train = shuffled.sample(frac = 0.8)
#     X_test = shuffled.drop(X_train_n.index)
#     y_test = X_test[["cardio"]]
#     X_test = X_test.drop(["cardio"], axis=1)
#     print(X_train)
    
#     errors = []
#     oner_results = []
#     for i, t in enumerate(["smoke", "alco", "active", "gender", "above", "gluc"]):
#         df_temp = X_train_n[[t, "cardio"]]
#         print(df_temp)
#         feature_params = df_temp[t].unique()
#         df_temp = df_temp.to_numpy().tolist()
#         fp_counts = []
#         for f in feature_params:
#             count = sum(map(lambda x : x[0] == f and x[1] == 1, df_temp))
#             print(f)
#             print()
#             one_count = count((df_temp[0] == f & df_temp[1] == 1), axis = 0)
#             zero_count = count((df_temp[0] == f & df_temp[1] == 0), axis = 0)
#             fp_counts.append([f, one_count, zero_count])
#         oner_results.append(fp_counts)
        
#     for o in oner_results:
#         error = [0, 0, 0]
#         error[0] = o[0][0]
#         for oo in o:
#             e = min(oo[1], oo[2])
#             index = oo.index(e)
#             if index:
#                 errors.append([o[0], 1, e/sum(o[1], o[2])])
#             else:
#                 errors.append([o[0], 0, e/sum(o[1], o[2])])
    
#     min_error = min(errors, key=lambda x: x[2])
#     print("min error:", min_error) 
    
#     error_rate = 0
#     for i in range(len(X_test)):
#         #print("predicted:", predict(min_error, X_test[i]), "actual:", y_test[i])
#         if predict_nominal(min_error, X_test[i]) != y_test[i]:
#             error_rate += 1
#     error_rate /= len(y_test)
#     error_rate_sum += error_rate
#     print("error_rate:", error_rate)
    
    
print("avarage error rate:", error_rate_sum/5)